In [1]:
#data context:
"""
https://www.kaggle.com/datasets/brunogrisci/breast-cancer-gene-expression-cumida
    
"""




import os 

os.getcwd()
dir_ = 'C://Users//Domin/Downloads'
os.chdir(dir_)

os.getcwd()
import pandas as pd
import numpy 
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('breast_cancer_bioinformatics.csv')

#lets do a little exploratory analysis 

data.dropna(axis = 1)
data.head()

#Over 16,000 genes !!! 

,type,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,206926_s_at,206927_s_at,206928_at,206929_s_at,206930_at,206931_at,206932_at,206933_s_at,206934_at,206935_at
0,basal,9.850040,8.097927,6.424728,7.353027,3.029122,6.880079,4.963740,4.408328,8.870780,...,6.167506,4.157291,8.463251,8.233774,3.440250,3.249266,4.077760,5.114589,3.649391,3.048597
1,basal,9.861357,8.212222,7.062593,7.685578,3.149468,7.542283,5.129607,4.584418,7.767646,...,6.016303,3.746938,9.166471,7.922956,2.832078,3.563185,4.457529,4.774928,3.886136,4.945811
2,basal,10.103478,8.936137,5.735970,7.687822,3.125931,6.562369,4.813449,4.425195,9.417956,...,6.298974,3.818161,8.691060,7.209782,3.371737,3.500604,4.909135,4.619495,3.393670,2.940155
3,basal,9.756875,7.357148,6.479183,6.986624,3.181638,7.802344,5.490982,4.567956,9.022345,...,5.924285,3.799996,8.519125,7.011523,3.412964,3.716722,4.822208,4.792745,3.619956,2.710000
4,basal,9.408330,7.746404,6.693980,7.333426,3.169923,7.610457,5.372469,4.424426,9.400056,...,5.984826,4.005609,9.199242,7.460834,3.155281,4.028018,5.705145,4.477804,3.724393,2.829142


In [3]:


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 
labeler = LabelEncoder()
y = data['type'] 
x = data.drop(['type'], axis = 1) 
x_copy = x 
y_copy = y


scaler = StandardScaler()
y.value_counts()

#six dimensions 

basal        41
HER          30
luminal_B    30
luminal_A    29
cell_line    14
normal        7
Name: type, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 24)

y_train = labeler.fit_transform(y_train)
y_test = labeler.fit_transform(y_test)



In [5]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_train_copy = X_train
y_train_copy = y_train


X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [6]:
#modified version - borrowed some code and modified for the functions


class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim,300)
        self.hidden_layer1  = nn.Linear(300,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out
    


In [7]:

input_dim  = 16382
output_dim = 6
model = NeuralNetworkClassificationModel(input_dim,output_dim)

In [8]:
# creating our optimizer and loss function object
learning_rate = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [9]:
import numpy as np 

In [10]:
def train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(X_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train)
        


        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        
        output_test = model(X_test)
        loss_test = criterion(output_test,y_test)

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")

In [11]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)
test_losses  = np.zeros(num_epochs)

In [12]:
train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses)

Epoch 50/1000, Train Loss: 0.0019, Test Loss: 0.2339
Epoch 100/1000, Train Loss: 0.0009, Test Loss: 0.2221
Epoch 150/1000, Train Loss: 0.0006, Test Loss: 0.2180
Epoch 200/1000, Train Loss: 0.0004, Test Loss: 0.2156
Epoch 250/1000, Train Loss: 0.0003, Test Loss: 0.2136
Epoch 300/1000, Train Loss: 0.0003, Test Loss: 0.2120
Epoch 350/1000, Train Loss: 0.0002, Test Loss: 0.2107
Epoch 400/1000, Train Loss: 0.0002, Test Loss: 0.2099
Epoch 450/1000, Train Loss: 0.0001, Test Loss: 0.2091
Epoch 500/1000, Train Loss: 0.0001, Test Loss: 0.2086
Epoch 550/1000, Train Loss: 0.0001, Test Loss: 0.2083
Epoch 600/1000, Train Loss: 0.0001, Test Loss: 0.2081
Epoch 650/1000, Train Loss: 0.0001, Test Loss: 0.2078
Epoch 700/1000, Train Loss: 0.0001, Test Loss: 0.2076
Epoch 750/1000, Train Loss: 0.0001, Test Loss: 0.2074
Epoch 800/1000, Train Loss: 0.0001, Test Loss: 0.2071
Epoch 850/1000, Train Loss: 0.0000, Test Loss: 0.2068
Epoch 900/1000, Train Loss: 0.0000, Test Loss: 0.2065
Epoch 950/1000, Train Loss: 0

In [13]:
predictions_train = []
predictions_test =  []
with torch.no_grad():
    predictions_train = model(X_train)
    predictions_test = model(X_test)

In [14]:
def get_accuracy_multiclass(pred_arr,original_arr):
    if len(pred_arr)!=len(original_arr):
        return False
    pred_arr = pred_arr.numpy()
    original_arr = original_arr.numpy()
    final_pred= []
  
    for i in range(len(pred_arr)):
        final_pred.append(np.argmax(pred_arr[i]))
    final_pred = np.array(final_pred)
    count = 0
    #here we are doing a simple comparison between the predicted_arr and the original_arr to get the final accuracy
    for i in range(len(original_arr)):
        if final_pred[i] == original_arr[i]:
            count+=1
    return count/len(final_pred)

In [15]:
train_acc = get_accuracy_multiclass(predictions_train,y_train)
test_acc  = get_accuracy_multiclass(predictions_test,y_test)

In [16]:
print(f"Training Accuracy: {round(train_acc*100,3)}")
print(f"Test Accuracy: {round(test_acc*100,3)}")

Training Accuracy: 100.0
Test Accuracy: 93.548


In [17]:
#nice, we got 93% accuracy! 

In [18]:
#lets see what we would have gotten with a Logistic Regression algorithm 

In [19]:
X_train = X_train.numpy()
X_test = X_test.numpy()
y_train = y_train.numpy()
y_test = y_test.numpy()

In [20]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


In [21]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

C:\Users\Domin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:

y_pred = log_model.predict(X_test)

print('\nModel Score: ',log_model.score(X_test, y_test),'\n')


Model Score:  0.9354838709677419 



In [23]:
#interesting - it got a very similar score! 


# get importance - if possible
importance = log_model.coef_[0]
gene_names = x.columns
gene_list, importance_list = [], []
    
for i,v in enumerate(importance):
    gene_list.append(x.columns[i])
    importance_list.append(importance[i])

In [24]:
df = pd.DataFrame()
df

""


In [25]:
df['Genes'] = gene_list
df['Importance'] = importance
df.head()

,Genes,Importance
0,1007_s_at,0.001400
1,1053_at,0.000018
2,117_at,-0.003272
3,121_at,0.001003
4,1255_g_at,0.001907


In [26]:
#now lets get the ten most 'important' genes 
df = df.sort_values(by = 'Importance', ascending = False)
df_copy = df
#now let's snip these genes 

df = df.iloc[0:10]
gene_list = df['Genes'].tolist()

gene_list



['202991_at',
 '201207_at',
 '1559697_a_at',
 '1566968_at',
 '1565635_at',
 '204323_x_at',
 '206793_at',
 '204909_at',
 '1560556_a_at',
 '204325_s_at']

In [27]:
x_copy_updated = x_copy[['202991_at',
 '201207_at',
 '1559697_a_at',
 '1566968_at',
 '1565635_at',
 '204323_x_at',
 '206793_at',
 '204909_at',
 '1560556_a_at',
 '204325_s_at']]

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_copy_updated, y, test_size = 0.20, random_state = 24)

In [29]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

C:\Users\Domin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
y_pred = log_model.predict(X_test)

print('\nModel Score: ',log_model.score(X_test, y_test),'\n')


Model Score:  0.7096774193548387 



In [31]:
#Not super great - but definitely not terrible. 70% accuracy with only ten of the thousands of other genes!
#Seems we may be getting diminising returns? 
#Let's see what would happen if we chose the top 25 instead 


top_25_genes = df_copy.iloc[0:25]
top_25_genes.head()

,Genes,Importance
12440,202991_at,0.009588
10655,201207_at,0.008557
4951,1559697_a_at,0.008429
8537,1566968_at,0.008417
7911,1565635_at,0.008409


In [32]:
top_25_list = top_25_genes['Genes'].tolist()
updated_df = x_copy

updated_df_25 = updated_df[top_25_list]
updated_df_25.head()

#excellent! now lets see by how much this improves the score (if any) 




,202991_at,201207_at,1559697_a_at,1566968_at,1565635_at,204323_x_at,206793_at,204909_at,1560556_a_at,204325_s_at,...,204915_s_at,205365_at,206165_s_at,1552801_at,1559696_at,206166_s_at,206199_at,1556923_at,201208_s_at,201388_at
0,5.989758,7.870343,3.732465,6.518358,4.014064,6.250654,5.486211,4.852879,3.327970,4.808402,...,6.959162,3.831934,3.099799,3.573151,4.169292,3.382761,3.408623,6.260609,7.047056,8.484028
1,6.138559,7.757607,3.522690,6.697095,4.816689,6.349760,5.315597,4.510545,4.133971,4.049191,...,6.502029,3.350953,3.383245,3.668351,3.947331,3.198395,3.386591,5.191695,6.712379,8.095460
2,6.014003,7.331337,3.358369,5.381590,3.731272,6.584204,5.110619,4.627865,3.298836,5.708482,...,3.918706,3.370024,3.222324,4.382866,3.951010,3.547763,3.329935,4.152518,6.372206,8.249896
3,6.546919,7.664019,3.362631,6.177053,3.488158,5.997989,5.496236,4.502260,3.294319,4.257647,...,6.249711,3.461685,3.781336,3.679822,3.730990,3.632173,3.020432,5.986792,6.681212,9.538022
4,6.596304,8.104621,3.477564,6.382301,4.033350,6.752754,5.494223,4.662594,3.922545,4.859110,...,6.054351,3.522720,3.039264,3.605025,3.658386,3.311291,3.270831,5.424712,7.008956,9.764372


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(updated_df_25, y, test_size = 0.20, random_state = 24)


In [34]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

C:\Users\Domin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [35]:
y_pred = log_model.predict(X_test)

print('\nModel Score: ',log_model.score(X_test, y_test),'\n')

# wow, it turns out that only 25 of these genes are sufficient in achieving 90% accuracy! 


Model Score:  0.9032258064516129 



In [36]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#man it looks like we are really missing out on cell line and normal - maybe 
#our algorithm needs some tuning/updating? 

#will return with an updated version to handle the class imbalances. 

[[5 0 0 0 0 0]
 [0 8 0 0 0 0]
 [0 1 0 0 0 1]
 [0 0 0 9 0 1]
 [0 0 0 0 6 0]
 [0 0 0 0 0 0]]
              precision    recall  f1-score   support

         HER       1.00      1.00      1.00         5
       basal       0.89      1.00      0.94         8
   cell_line       0.00      0.00      0.00         2
   luminal_A       1.00      0.90      0.95        10
   luminal_B       1.00      1.00      1.00         6
      normal       0.00      0.00      0.00         0

    accuracy                           0.90        31
   macro avg       0.65      0.65      0.65        31
weighted avg       0.91      0.90      0.90        31



C:\Users\Domin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Domin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Domin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Domin\anaconda3\lib\site-packages\sklearn\metrics\_cla